In [1]:
import utils.conf as conf
import preprocessing.data_cleaner as cleaner
import preprocessing.feature_extractor as extractor
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2

In [2]:
train, _ = cleaner.clean()


In [ ]:
features = extractor.aggregate_features_v2(train)
train.to_csv('train_features.csv', sep=',')

0 of 3584 fineline number features have been added.
100 of 3584 fineline number features have been added.
200 of 3584 fineline number features have been added.
300 of 3584 fineline number features have been added.
400 of 3584 fineline number features have been added.
500 of 3584 fineline number features have been added.

In [3]:
features = pd.read_csv('train_features.csv', sep=',')

In [4]:
features.head()

,VisitNumber,fn_-1,fn_0,fn_1,fn_2,fn_3,fn_4,fn_5,fn_6,fn_7,...,SEAFOOD,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS
0,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
2,8,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
3,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
from sklearn.model_selection import train_test_split
y = train.groupby(['VisitNumber'])['TripType'].first()
X = features
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=32)

In [17]:
X_train.drop('TripType', axis=1, inplace=True)
X_test.drop('TripType', axis=1, inplace=True)

/home/wei/anaconda2/envs/pt27/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/wei/anaconda2/envs/pt27/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [5]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, n_jobs=8)
clf.fit(X_train,y_train)
clf_probs = clf.predict_proba(X_test)


importances = clf.feature_importances_
std = np.std([tree.feature_importances_ for tree in clf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")
for f in range(X_train.shape[1]):
    print("%d. feature %d : %s (%f)" % (f + 1, indices[f], X_train.columns[indices[f]],importances[indices[f]]))
#from sklearn.model_selection import cross_val_score
#scores = cross_val_score(clf, X_test, y_test.values)
scores = clf.score(X_test,y_test)
print(scores)

from sklearn.metrics import log_loss
logloss_score = log_loss(y_test, clf_probs)
print(logloss_score)

Feature ranking:
1. feature 3585 : ScanCountSum (0.058787)
2. feature 3590 : Purchased_num (0.050470)
3. feature 3588 : FinelineNumber_has_max (0.038792)
4. feature 3586 : Encoded_UPC_nunique (0.037919)
5. feature 0 : VisitNumber (0.032869)
6. feature 3608 : DSD GROCERY (0.020954)
7. feature 3589 : Returned_num (0.020698)
8. feature 3643 : PHARMACY OTC (0.017457)
9. feature 3641 : PERSONAL CARE (0.015990)
10. feature 3615 : GROCERY DRY GOODS (0.014790)
11. feature 3611 : FINANCIAL SERVICES (0.014286)
12. feature 3648 : PRODUCE (0.012890)
13. feature 3634 : MENS WEAR (0.011908)
14. feature 3651 : SERVICE DELI (0.011889)
15. feature 3587 : FinelineNumber_max_num (0.011668)
16. feature 3607 : DAIRY (0.011247)
17. feature 3621 : HOUSEHOLD CHEMICALS/SUPP (0.009177)
18. feature 3623 : IMPULSE MERCHANDISE (0.007813)
19. feature 3625 : INFANT CONSUMABLE HARDLINES (0.007776)
20. feature 3644 : PHARMACY RX (0.007451)
21. feature 3612 : FROZEN FOODS (0.007278)
22. feature 3596 : BEAUTY (0.007162)

**XGBoost**

In [18]:
import models.models as models
depth = 5
eta = 0.3
xgb_clf = models.XGBoostMutliClass(nthread=8, eta=eta, 
                                   max_depth=depth, num_round=50, silent=0, booster='gbtree')
xgb_clf.fit(X_train, y_train)

XGBoostMutliClass(booster='gbtree', eta=0.3, max_depth=5, nthread=8,
         num_round=50, silent=0)

In [8]:
len(xgb_clf.model.get_fscore())

96

In [10]:
xgb_clf.model.get_fscore()

{'1-HR PHOTO': 3,
 'ACCESSORIES': 7,
 'AUTOMOTIVE': 35,
 'BATH AND SHOWER': 1,
 'BEAUTY': 6,
 'BEDDING': 5,
 'BOOKS AND MAGAZINES': 11,
 'BOYS WEAR': 1,
 'BRAS & SHAPEWEAR': 1,
 'CAMERAS AND SUPPLIES': 3,
 'CANDY, TOBACCO, COOKIES': 13,
 'CELEBRATION': 3,
 'COOK AND DINE': 11,
 'DAIRY': 15,
 'DSD GROCERY': 49,
 'ELECTRONICS': 22,
 'Encoded_UPC_nunique': 189,
 'FABRICS AND CRAFTS': 22,
 'FINANCIAL SERVICES': 1,
 'FURNITURE': 1,
 'FinelineNumber_has_max': 295,
 'FinelineNumber_max_num': 18,
 'GIRLS WEAR, 4-6X  AND 7-14': 8,
 'GROCERY DRY GOODS': 22,
 'HARDWARE': 21,
 'HOME DECOR': 10,
 'HOME MANAGEMENT': 11,
 'HORTICULTURE AND ACCESS': 20,
 'HOUSEHOLD CHEMICALS/SUPP': 5,
 'HOUSEHOLD PAPER GOODS': 9,
 'IMPULSE MERCHANDISE': 28,
 'INFANT APPAREL': 11,
 'INFANT CONSUMABLE HARDLINES': 9,
 'JEWELRY AND SUNGLASSES': 5,
 'LADIESWEAR': 8,
 'LAWN AND GARDEN': 23,
 'LIQUOR,WINE,BEER': 9,
 'MEAT - FRESH & FROZEN': 3,
 'MEDIA AND GAMING': 27,
 'MENS WEAR': 19,
 'OFFICE SUPPLIES': 15,
 'PAINT AND ACC

In [9]:
len(xgb_clf.model.feature_names)

3660

In [ ]:
yprob, preds = xgb_clf.predict_proba(X_test)
from sklearn.metrics import log_loss
logloss_score = log_loss(y_test, yprob)
print(logloss_score)

In [4]:
import models.models as models
for depth in [5, 10, 15]:
    for eta in [0.1, 0.15, 0.2]:
        xgb_clf = models.XGBoostMutliClass(nthread=8, eta=eta, max_depth=depth, num_round=50, silent=0)
        xgb_clf.fit(X_train, y_train)
        yprob, preds = xgb_clf.predict_proba(X_test)
        from sklearn.metrics import log_loss
        logloss_score = log_loss(y_test, yprob)
        with open('grid_search.txt','a') as out_file:
            out_file.write("eta=%.2f max_depth=%d log_loss=%f\n" %(eta, depth, logloss_score))

/home/wei/anaconda2/envs/pt27/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


* quantile=0.3 eta=0.3 depth=20 0.852055502871
* quantile=0.3 eta=0.1 depth=20 0.754730680399
* quantile=0.8 0.771723597482

**Submission**

In [1]:
train = extractor.aggregate_features_v2(train)
train.to_csv('train_features.csv', sep=',')
test = extractor.aggregate_features_v2(test)
test.to_csv('test_features.csv', sep=',')



NameError: name 'extractor' is not defined

In [ ]:
train = pd.read_csv('train_features.csv', sep=',')
test = pd.read_csv('test_features.csv', sep=',')
y_train = train['TripType']
for col_name in train.columns:
    if col_name not in test.columns:
        train.drop(col_name, axis=1, inplace=True)
for col_name in test.columns:
    if col_name not in train.columns:
        test.drop(col_name, axis=1, inplace=True)

In [9]:
train = pd.read_csv('train_features.csv', sep=',', nrows=2)

In [11]:
test = pd.read_csv('test_features.csv', sep=',')
for col_name in test.columns:
    if col_name not in train.columns:
        test.drop(col_name, axis=1, inplace=True)

In [24]:
train.shape

(2, 3660)

In [3]:
import models.models as models
xgb_clf = models.XGBoostMutliClass(nthread=8, eta=0.3, max_depth=10, num_round=100, silent=1)
X_train = train
xgb_clf.fit(X_train, y_train)


/home/wei/anaconda2/envs/pt27/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


XGBoostMutliClass(eta=0.1, max_depth=20, nthread=8, num_round=100, silent=1)

In [30]:
del train

NameError: name 'train' is not defined

In [12]:
import pickle
xgb_clf = pickle.load(open('xgb_clf.p','rb'))

/home/wei/anaconda2/envs/pt27/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [13]:
yprob, classes = xgb_clf.predict_proba(test)

column_names = ['VisitNumber']
for cls in classes:
    column_names.append('TripType_' + str(cls))
out = pd.DataFrame(columns=column_names)


In [20]:
out['VisitNumber'] = test['VisitNumber']
out.ix[:,'TripType_3':] = yprob
out.to_csv('submission.csv', sep=',', index=False)

In [23]:
out.head(50)

,VisitNumber,TripType_3,TripType_4,TripType_5,TripType_6,TripType_7,TripType_8,TripType_9,TripType_12,TripType_14,...,TripType_36,TripType_37,TripType_38,TripType_39,TripType_40,TripType_41,TripType_42,TripType_43,TripType_44,TripType_999
0,1,0.000059,0.000058,0.000085,0.000150,0.000700,0.008183,0.006694,0.000061,0.000139,...,0.000102,0.000065,0.298867,0.011419,0.000060,0.000668,0.000611,0.000831,0.000219,0.001452
1,2,0.000152,0.000150,0.000233,0.001158,0.013924,0.015368,0.017371,0.000158,0.000363,...,0.000285,0.000187,0.008059,0.120226,0.000155,0.000548,0.004800,0.003492,0.000324,0.052518
2,3,0.000021,0.000011,0.000019,0.000139,0.000167,0.000383,0.000220,0.000012,0.000027,...,0.000019,0.000012,0.000024,0.000018,0.000011,0.000050,0.000017,0.000012,0.000012,0.998524
3,4,0.000098,0.000087,0.000146,0.001077,0.002400,0.033524,0.875595,0.000092,0.000210,...,0.000152,0.000094,0.000192,0.000143,0.000090,0.000391,0.000146,0.000095,0.000092,0.060133
4,6,0.000025,0.000013,0.000020,0.000052,0.000134,0.000491,0.000259,0.000013,0.000031,...,0.000022,0.000014,0.000028,0.000021,0.000013,0.000057,0.000028,0.000014,0.000013,0.998390
5,13,0.000144,0.000138,0.000232,0.002168,0.244606,0.685594,0.046460,0.000146,0.000334,...,0.000241,0.000720,0.000347,0.000239,0.000144,0.000620,0.000193,0.000150,0.000146,0.006288
6,14,0.000192,0.000189,0.000271,0.001140,0.007028,0.000235,0.000275,0.000199,0.000457,...,0.000293,0.000566,0.000961,0.710892,0.002370,0.005114,0.014521,0.014678,0.094823,0.001159
7,16,0.000154,0.000152,0.000242,0.000806,0.004688,0.000263,0.000213,0.000160,0.000366,...,0.000275,0.004860,0.001519,0.070790,0.001875,0.000460,0.002730,0.014821,0.001978,0.000723
8,18,0.000217,0.000214,0.000326,0.002120,0.041904,0.000401,0.000574,0.000226,0.000517,...,0.000389,0.015637,0.010460,0.365173,0.001371,0.000703,0.001152,0.003763,0.001807,0.001616
9,21,0.000136,0.000972,0.029129,0.000345,0.000966,0.000199,0.000619,0.000142,0.000324,...,0.810661,0.000170,0.000581,0.100712,0.000920,0.002984,0.001617,0.012948,0.002097,0.002685
